# AIDAR Objective 1.0 Hypotheses Testing

We are leaning towards employing [Naive Bayes](https://kiranvoleti.com/how-to-use-a-naive-bayes-classifier-for-marketing-analytics) or similar classifier for our budget predictions (i.e. autoML with [PyCaret](https://medium.com/@atalarifqy/optimizing-machine-learning-workflows-with-pycaret-e0b19d8caf2c)). In order to confirm the associated and impacting features on the actions, we must first run a Chi-square test to validate the importance of the variables.

* Query a representative sample of the relevant dataset for each hypotheses
* Test the null and alternative hypotheses (P-value) for the confidence interval 
* Present the results for discussion and evaluation

[Fundermentals of Chi-Square Test](https://www.simplilearn.com/tutorials/statistics-tutorial/chi-square-test)

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


| Analytics Activity |Hypothese to test | Required data |
|--------|-----|--------------|
|AA-1.1.1.1 |Test whether Adsets with higher budget_remaining correlate with higher impressions but not necessarily with better ROI. |client type<br>ad_id<br>timestamp<br>remaining budget (allocated - spent)<br>impressions count<br>ROI (CTR, CPM, ...) |
|AA-1.1.1.2 |Discover all  other correlating factors that may contribute to underperforming adsets | |
|AA-1.2.1.1 |Consider all optimization_goal types (e.g., conversions) and their Powerset combinations on Adsets that outperform those targeting general engagement. | |
|AA-1.3.1.1 |Campaigns with defined bid_strategy types (e.g., cost cap) lead to lower CPA. | |
|AA-1.3.1.2 |Look at the statistically significant power sets of objective and strategy elements | |
|AA-1.4.1.1 |Test whether campaigns with a higher daily_budget allocated to peak hours achieve better ROI | |
|AA-1.4.1.2 |Calculate the timeframes that offer high impact time segments or seasonal effects (e.g. peak hours) | |
|AA-1.5.1.1 |Evaluate whether Ads targeting specific demographics yield better ROI than generic targeting. | |

## Instantiate Classes

In [2]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('mining/')[0])
from rezaware.modules.etl.loader import sparkRDBM as db
# from rezaware.modules.etl.loader import __propAttr__ as attr

''' restart initiate classes '''
if debug:
    import importlib
    db = importlib.reload(db)
    # attr=importlib.reload(attr)

__desc__ = "read and write BigQuery dataset for hypothese testing"
clsSDB = db.dataWorkLoads(
    desc=__desc__,
    db_type = 'bigquery',
    db_driver=None,
    db_hostIP=None,
    db_port = None,
    db_name = None,
    db_schema='combined_data_facebook_ads',
    spark_partitions=None,
    spark_format = 'bigquery',
    spark_save_mode=None,
    # spark_jar_dir = _jar,
)
# if clsSDB.session:
#     clsSDB._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional __PROPATTR__-libraries in LOADER-package of ETL-module imported successfully!
All functional SPARKRDBM-libraries in LOADER-package of ETL-module imported successfully!
All functional SPARKRDBM-libraries in LOADER-package of ETL-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
__propAttr__ Class initialization complete

read and write BigQuery dataset for hypothese testing class initialization and load complete!


## Load data
There are two options for loading the data
1. querying with a select statement by defining the sql _query_ statement string
2. reading an entire table with row limit by defining the _tbl_ comprising _project:dataset.table_

By default the function _read_data_from_table() will check for the select variable; else will revert to the _tbl description to read the data
Use either or; i.e.,: 
* if using the select query set ```db_table = None``` and assign ```select=query```
* if using db_table, then set ```select = None``` and assign ```db_table = _tbl```

In [3]:
query = """
    SELECT id, name, status, campaign, end_time, estimate, dsa_payor, targeting,
        account_id, bid_amount, start_time, campaign_id, created_time, daily_budget,
        source_adset, updated_time, billing_event, lifetime_imps, dsa_beneficiary, 
        lifetime_budget, promoted_object, review_feedback, source_adset_id, budget_remaining,
        destination_type, effective_status, configured_status, optimization_goal,
        use_new_app_click, is_dynamic_creative, campaign_active_time, campaign_attribution,
        optimization_sub_event, recurring_budget_semantics, min_budget_spend_percentage,
        multi_optimization_goal_weight
    FROM fresh-deck-445306-b0.combined_data_facebook_ads.combine__metadata_adset as adset
    LIMIT 1000
"""
_tbl = "fresh-deck-445306-b0:combined_data_facebook_ads.cl_act__ads_insights_action_values"
options = {
    "project" : 'fresh-deck-445306-b0',
    "dataset" : 'combined_data_facebook_ads',
}
sdf = clsSDB.read_data_from_table(
    select=None, #query,
    db_table=_tbl,
    **options
)
print("Loaded %d rows" % sdf.count())
sdf.printSchema()

25/01/15 18:22:40 WARN Utils: Your hostname, Waidy-Think-Three resolves to a loopback address: 127.0.1.1; using 192.168.126.57 instead (on interface wlp3s0)
25/01/15 18:22:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/01/15 18:22:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
                                                                                

Loaded 172689 rows
root
 |-- ad_id: string (nullable = true)
 |-- _airbyte_ab_id: string (nullable = true)
 |-- updated_time: string (nullable = true)
 |-- json_array: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- d1_click: double (nullable = true)
 |-- d7_click: double (nullable = true)
 |-- d28_click: double (nullable = true)
 |-- d1_click_first_conversion: double (nullable = true)
 |-- d7_click_first_conversion: double (nullable = true)
 |-- d28_click_first_conversion: double (nullable = true)
 |-- d1_click_alll_conversion: double (nullable = true)
 |-- d7_click_all_conversion: double (nullable = true)
 |-- d28_click_allt_conversion: double (nullable = true)
 |-- d1_view: double (nullable = true)
 |-- d7_view: double (nullable = true)
 |-- d28_view: double (nullable = true)
 |-- d1_view_first_conversion: double (nullable = true)
 |-- d7_view_first_conversion: double (nullable = true)
 |-- d28_view_first_conversion: double (nullable = true)
 |-- d1_view

## Filter dataframe by selected columns
__Inputs__
* set the ```_pop_id_col``` variable with an identifier (not necessarilly unique)
* list the ```_filter_by_cols``` with the required set of column names
__outputs__
* ```filt_sdf_``` will produce the filtered dataframe such that none of the rows contain a column with Null value

In [4]:
from pyspark.sql import functions as F

_pop_id_col = 'ad_id'
_filter_by_cols = ['ad_id', 'd1_click', 'd7_click', 'd28_click', 'd1_view', 'd7_view', 'd28_view', 
                   'action_type', 'value']
filt_sdf_ = sdf.select(_filter_by_cols).dropna()
print("Total filtered rows %d for %d unique %s values" 
      % (filt_sdf_.count(), filt_sdf_.select(F.col(_pop_id_col)).distinct().count(), _pop_id_col.upper()))
filt_sdf_.show(n=5, truncate=False)

[Stage 9:>                                                          (0 + 1) / 1]

Total filtered rows 172689 for 4906 unique AD_ID values


[Stage 15:>                                                         (0 + 1) / 1]

+-------------+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+
|ad_id        |d1_click|d7_click|d28_click|d1_view|d7_view|d28_view|action_type                              |value |
+-------------+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+
|6216259122941|69.46   |305.04  |305.04   |160.31 |285.95 |490.8   |offsite_conversion.fb_pixel_add_to_cart  |465.35|
|6216259122941|115.33  |168.53  |168.53   |54.52  |273.93 |273.93  |offsite_conversion.fb_pixel_purchase     |223.05|
|6216259122941|0.0     |10.45   |10.45    |0.0    |0.0    |0.0     |offsite_conversion.fb_pixel_custom       |10.45 |
|6216259122941|17.0    |55.5    |55.5     |847.77 |1975.11|2084.1  |add_to_cart                              |903.27|
|6216259122941|0.0     |0.0     |0.0      |6.16   |6.16   |6.16    |offsite_conversion.custom.814730377326332|6.16  |
+-------------+--------+--------+---------+-------+-----

## Randomly sample the data
[Calculating your sample size](https://www.remesh.ai/resources/how-to-calculate-sample-size)
#### Confidence interval
* First we set the deisred _confidence level_:
  * also cointed as margin of error; typically set at 5%
  * will result in a 95% confindece interval (can also use 90% and 99%)
  * hence, we set ```alpha=0.95```
* _Tolerance_ interval: 90% (0.90 percentile) ensures that p% of the sample
* _Prediction_ interval
* Define the _Population_ by ramdomly selecting N representative rows

In [5]:
from pyspark.sql import functions as F

def confidence_interval(df, grp):
    # col = F.when(F.col('status') == F.lit('ACTIVE'), 1.0).otherwise(0.0)
    col_ = 'd1_click'

    return df.groupBy(grp).agg(
        F.mean(col_).alias('mean'),
        F.count(F.col(col_)).alias('total'),
        F.stddev(col_).alias('std')
    ).withColumn(
        'ci95_hi',
        F.col('mean') + 1.96 * F.col('std') / F.sqrt(F.col('total'))
    ).withColumn(
        'ci95_lo',
        F.col('mean') - 1.96 * F.col('std') / F.sqrt(F.col('total'))
    )

confidence_interval(filt_sdf_, 'action_type').show(truncate=False)

[Stage 16:>                                                         (0 + 1) / 1]

+---------------------------------------------+------------------+-----+------------------+------------------+------------------+
|action_type                                  |mean              |total|std               |ci95_hi           |ci95_lo           |
+---------------------------------------------+------------------+-----+------------------+------------------+------------------+
|offsite_conversion.fb_pixel_search           |640.7014705882353 |340  |870.6589685522944 |733.2489704276998 |548.1539707487709 |
|offsite_conversion.custom.814730377326332    |0.7744172086043025|1999 |2.5929290250512023|0.8880857542863496|0.6607486629222554|
|offsite_conversion.custom.6508804492500376   |36.32727272727273 |55   |34.51197030417264 |45.44831525517694 |27.206230199368523|
|offsite_conversion.fb_pixel_purchase         |135.0893611937763 |69896|8289.946985837309 |196.54781971682343|73.6309026707292  |
|offsite_conversion.fb_pixel_custom           |23.921662269129286|379  |60.276364137898845

In [5]:
column_index_1 = 'd28_click'
column_index_2 = 'd28_view'

fraction = 0.4
''' Apply transformation on every element by defining the columns (first, second) 
    as well as sampling percentage as an argument in the map function '''
fractions = filt_sdf_.rdd.map(lambda x:
                              (x[column_index_1],
                               x[column_index_2]))\
                    .distinct()\
                    .map(lambda x:
                         (x,fraction)).collectAsMap()
print("created franctions for %d index pairs" % len(fractions))
# Create tuple of elements using keyBy function 
key_df = filt_sdf_.rdd.keyBy(lambda x: (x[column_index_1],x[column_index_2]))

# Extract random sample through sampleByKey function 
# using boolean, columns (first and second) and fraction as arguments 
print("Sample 1: ")
sample_1_sdf = key_df.sampleByKey(
    False, fractions).map(lambda x: x[1]).toDF(filt_sdf_.columns)
sample_1_sdf.show(n=5, truncate=False)
 
# Again extract random sample through sampleByKey function 
# using boolean, columns (first and second) and fraction as arguments 
print("Sample 2: ")
sample_2_sdf = key_df.sampleByKey(
    False, fractions).map(lambda x: x[1]).toDF(filt_sdf_.columns)
sample_2_sdf.show(n=5, truncate=False)

created franctions for 73070 index pairs
Sample 1: 


+-------------+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+
|ad_id        |d1_click|d7_click|d28_click|d1_view|d7_view|d28_view|action_type                              |value |
+-------------+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+
|6216259122941|69.46   |305.04  |305.04   |160.31 |285.95 |490.8   |offsite_conversion.fb_pixel_add_to_cart  |465.35|
|6216259122941|115.33  |168.53  |168.53   |54.52  |273.93 |273.93  |offsite_conversion.fb_pixel_purchase     |223.05|
|6216259122941|0.0     |0.0     |0.0      |6.16   |6.16   |6.16    |offsite_conversion.custom.814730377326332|6.16  |
|6216259122941|150.8   |150.8   |150.8    |317.55 |530.58 |530.58  |offsite_conversion.fb_pixel_purchase     |468.35|
|6216259324541|85.73   |290.66  |290.66   |395.09 |486.82 |505.65  |offsite_conversion.fb_pixel_purchase     |685.75|
+-------------+--------+--------+---------+-------+-----

[Stage 21:>                                                         (0 + 1) / 1]

+-------------+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+
|ad_id        |d1_click|d7_click|d28_click|d1_view|d7_view|d28_view|action_type                              |value |
+-------------+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+
|6216259122941|69.46   |305.04  |305.04   |160.31 |285.95 |490.8   |offsite_conversion.fb_pixel_add_to_cart  |465.35|
|6216259122941|115.33  |168.53  |168.53   |54.52  |273.93 |273.93  |offsite_conversion.fb_pixel_purchase     |223.05|
|6216259122941|17.0    |55.5    |55.5     |847.77 |1975.11|2084.1  |add_to_cart                              |903.27|
|6216259122941|0.0     |0.0     |0.0      |6.16   |6.16   |6.16    |offsite_conversion.custom.814730377326332|6.16  |
|6216259122941|26.24   |91.7    |91.7     |634.78 |697.34 |697.34  |offsite_conversion.fb_pixel_purchase     |726.48|
+-------------+--------+--------+---------+-------+-----

## Chi-square test

#### Apply OneHotEncoding
Convert categorical strings to numerical values

In [6]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.stat import ChiSquareTest

''' define sample data '''
sample_sdf = sample_1_sdf .drop(F.col('ad_id'))

''' StringIndexer Initialization '''
_label_col = 'action_type'
indexer = StringIndexer(inputCol=_label_col, outputCol="Categories_Indexed")
indexerModel = indexer.fit(sample_sdf)

# Transform the DataFrame using the fitted StringIndexer model
indexed_df = indexerModel.transform(sample_sdf)
# indexed_df.show()
''' apply encoder '''
encoder = OneHotEncoder(inputCol="Categories_Indexed", outputCol="labels")
encoded_df = encoder.fit(indexed_df).transform(indexed_df)
encoded_df.select(_label_col, "Categories_Indexed", "labels").show(n=5, truncate=False)

[Stage 25:>                                                         (0 + 1) / 1]

+-----------------------------------------+------------------+--------------+
|action_type                              |Categories_Indexed|labels        |
+-----------------------------------------+------------------+--------------+
|offsite_conversion.fb_pixel_add_to_cart  |1.0               |(16,[1],[1.0])|
|offsite_conversion.fb_pixel_purchase     |0.0               |(16,[0],[1.0])|
|offsite_conversion.custom.814730377326332|6.0               |(16,[6],[1.0])|
|offsite_conversion.fb_pixel_purchase     |0.0               |(16,[0],[1.0])|
|offsite_conversion.fb_pixel_purchase     |0.0               |(16,[0],[1.0])|
+-----------------------------------------+------------------+--------------+
only showing top 5 rows



In [7]:
_remove_lst = ['action_type', 'labels']
_feature_cols = [x for x in encoded_df.columns if x not in _remove_lst]
# _features='d28_click'

assembler = VectorAssembler(inputCols = _feature_cols, outputCol = 'features')
vec_sdf = assembler.transform(encoded_df)
vec_sdf.show(n=5, truncate=False)


[Stage 26:>                                                         (0 + 1) / 1]

+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+------------------+--------------+-----------------------------------------------------+
|d1_click|d7_click|d28_click|d1_view|d7_view|d28_view|action_type                              |value |Categories_Indexed|labels        |features                                             |
+--------+--------+---------+-------+-------+--------+-----------------------------------------+------+------------------+--------------+-----------------------------------------------------+
|69.46   |305.04  |305.04   |160.31 |285.95 |490.8   |offsite_conversion.fb_pixel_add_to_cart  |465.35|1.0               |(16,[1],[1.0])|[69.46,305.04,305.04,160.31,285.95,490.8,465.35,1.0] |
|115.33  |168.53  |168.53   |54.52  |273.93 |273.93  |offsite_conversion.fb_pixel_purchase     |223.05|0.0               |(16,[0],[1.0])|[115.33,168.53,168.53,54.52,273.93,273.93,223.05,0.0]|
|0.0     |0.0     |0.0      |6.16   |6.1

In [26]:
r = ChiSquareTest.test(vec_sdf.select('features', 'Categories_Indexed').limit(100), 'features', 'Categories_Indexed')

## Results
the chi-square test result helps you determine if differences in feature values are due to chance or if they reflect a true pattern. It is used to test and compare two different types of data: observed data and expected data; thus, measures what is called the "goodness to fit" which is the difference between what you'd expect and what has been observed

The hypothese we are testing is to determine if the features: ```d1_click, d7_click, d28_click, d1_view, d7_view, d28_view, value``` have an effect (or are associated) with ```action_type```; which may describe how budgets might be distributed???)

w.r.t Chi-square test:
*  _P-Value_ is a statistical measure that helps to assess the importance of your test results
*  _ndegreesOfFreedom_ is the number of cells in the deature sample that can vary before the rest can be calculated
*  _statistic_ compares the size of any discrepancies between the expected results and the actual results, given the size of the sample and the number of variables in the relationship

In [41]:
vals = r.first()
print("Sample size: 1000")
print("\nfeatures: d1_click, d7_click, d28_click, d1_view, d7_view, d28_view, value")
print("label: action_type")
print("\npValues: " + str(vals[0]))
# print("\ndegreesOfFreedom: " + str(vals[1]))
# print("\nstatistics: " + str(vals[2]))

Sample size: 1000

features: d1_click, d7_click, d28_click, d1_view, d7_view, d28_view, value
label: action_type

pValues: [0.048493226111149035,0.44627209161354975,0.6508140984033642,0.6624499875134036,0.46458059439628785,0.625566214596088,0.31327071857175204,0.0]


## DEPRECATED

In [21]:
import pandas as pd
from scipy.stats import chi2_contingency
from pyspark.mllib.stat import Statistics as stat

sample_sdf = vec_sdf

col_1 = 'd28_click'
col_2 = 'Categories_Indexed'

chisqr_sdf = sample_sdf.select(col_1,col_2)\
                .where((F.col(col_1).isNotNull()) & (F.col(col_1)==0))\
                .where((F.col(col_2).isNotNull()) & (F.col(col_2)==0))

contingency_table = chisqr_sdf.stat.crosstab(col_1, col_2)
contingency_table.show(truncate=False)

#Converting spark contingency_table to pandas DataFrame
contingency_table_df = contingency_table.toPandas()
contingency_table_df = contingency_table_df.set_index(col_1+"_"+col_2)
# Perform the chi-square test
chi2, p_value, degrees_of_freedom, expected_frequencies = chi2_contingency(contingency_table_df)

# Print the results
print("Chi-Square Statistic:", chi2)
print("P-Value:", p_value)
print("Degrees of Freedom:", degrees_of_freedom)

print(" ")
print("Contingency Table:")
print(contingency_table_df)

print(" ")
print("Expected Frequencies:")
print(pd.DataFrame(expected_frequencies, index=contingency_table_df.index, columns=contingency_table_df.columns))

+----------------------------+-----+
|d28_click_Categories_Indexed|0.0  |
+----------------------------+-----+
|0.0                         |12353|
+----------------------------+-----+



[Stage 47:>                                                         (0 + 1) / 1]

Chi-Square Statistic: 0.0
P-Value: 1.0
Degrees of Freedom: 0
 
Contingency Table:
                                0.0
d28_click_Categories_Indexed       
0.0                           12353
 
Expected Frequencies:
                                  0.0
d28_click_Categories_Indexed         
0.0                           12353.0


In [ ]:
sample_sdf = sdf.select(_filter_by_cols)\
                .where(F.col('click_d28').isNotNull())\
                .orderBy(F.rand()).limit(100)
print("randomly selected %d rows" % sample_sdf.count())
sample_sdf.show(n=3)

randomly selected 100 rows


[Stage 56:>                                                         (0 + 1) / 1]

+---------+------+
|d28_click| value|
+---------+------+
|     45.0|  45.0|
|      0.0|215.49|
|   345.23|456.38|
+---------+------+
only showing top 3 rows

